In [1]:
import surp
import vice
from vice.yields import ccsne, agb, sneia

/users/PAS2232/aeyobd/VICE/vice/__init__.py:141: UserWarning: Using un-released version of VICE
  warnings.warn("Using un-released version of VICE", UserWarning)


In [2]:
surp.set_yields()

yields set to Magg et al. 2022 abundances
Yield settings
X        Z_solar    CC                             agb                            SN Ia                          

c        3.39e-03   2.28e-03 + 0.00e+00 (Z - Z0)   1.00 × cristallo11             0                              

n        1.04e-03   5.00e-04                       5.02e-04 M Z/Z0                0                              

o        7.33e-03   7.13e-03                       0                              0                              

mg       6.71e-04   6.52e-04                       0                              0                              

fe       1.37e-03   4.73e-04                       0                              7.70e-04                       





# inputs from mcmc fits

In [3]:
Y_C_AGB = {
    "cristallo11": 3.8e-4,
    "ventura13": 1.85e-4,
    "karakas16": 2.8e-4,
    "pignatari16": 5.9e-4
}

In [4]:
ZETA_C_AGB = {
    "cristallo11": -3.5e-4,
    "ventura13": -9.4e-4,
    "karakas16": -10.1e-4,
    "pignatari16": -5.7e-4
}

# inputs from equilibrium fits

In [5]:
y_mg = vice.yields.ccsne.settings["mg"]
noneq_factor = 1.025
noneq_zeta = 1.8
noneq_A = 1.8

Y_C_0 = 4.12 * y_mg * noneq_factor
ZETA_C_0 = 1.21 * y_mg * noneq_factor * noneq_zeta
A_C_0 = 3.07 * y_mg * noneq_factor * noneq_A

# yield params calculator

Given the desired f_agb or alpha, what are the required y0cc and alpha_agb values

In [11]:
def f_agb_to_alpha(study, f_agb):
    y_agb0 = Y_C_AGB[study]
    y_agb = Y_C_0 * f_agb
    return y_agb / y_agb0

In [17]:
import json

In [37]:
def f_agb_to_settings(study, f_agb):
    alpha = f_agb_to_alpha(study, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(json.dumps(settings, indent=4))

In [36]:
def alpha_to_settings(study, alpha):
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(json.dumps(settings, indent=4))

In [38]:
0.0022027 - 0.00011117

0.0020915300000000003

In [35]:
f_agb_to_settings("cristallo11", 1)

{
    "alpha_c_agb": 7.245778947368421,
    "y0_c_cc": 0.0
}


In [6]:

def print_yc_tot():
    """prints out the current total c yield in an (almost nice) table"""

    print('total c yield')

    ycc = ccsne.settings["c"]
    yagb = agb.settings["c"]

    y0_cc = ycc.y0
    zeta_cc = ycc.zeta

    if isinstance(yagb, agb.interpolator):
        model = yagb.study
        alpha = yagb.prefactor
        y0_agb = alpha * Y_C_AGB[model]
        zeta_agb = alpha * ZETA_C_AGB[model]
        print("y0_agb ", Y_C_AGB[model])
        print("z0_agb ", ZETA_C_AGB[model])
    elif isinstance(yagb, C_AGB_Model):
        model = "A"
        y0_agb = yagb.y0
        zeta_agb = yagb.zeta
        print("Y agb", y0_agb)
        print("zeta agb", zeta_agb)

    print("agb_model: ", model)

    z_tot = zeta_cc + zeta_agb
    y_tot =  y0_agb + y0_cc
    f = y0_agb/y_tot

    print(f"{y_tot:0.6f} + {z_tot:0.6f} (Z-Zo)")
    print(f"f_agb = {f:0.4f}")
    print()
    print()

In [7]:
def set_c_cc(params, y0, zeta, y_c_cc="Lin", y1=1e-4, Z1=0):
    params.y0_c_cc = y0
    params.zeta_c_cc = zeta
    params.y_c_cc= y_c_cc
    if y_c_cc == "BiLin":
        params.kwargs_c_cc = dict(y1=y1, Z1=Z1)
    elif y_c_cc == "BiLogLin":
        params.kwargs_c_cc = dict(y1=y1)
    elif y_c_cc == "Quadratic":
        params.kwargs_c_cc = dict(A=A_C_0)


In [8]:
def set_c_agb(params, agb_model="cristallo11", f_agb=0.2, alpha_agb=None, zeta_agb=None, 
            mass_factor=1, no_negative=False, interp_kind="linear", t_D=0.1, tau_agb=0.3, low_z_flat=False):

    y0 = f_agb * Y_C_0
    params.Y_c_agb = agb_model

    if agb_model == "A":
        if zeta_agb is None:
            raise ValueError("for analytic AGB model, zeta_agb must be specified")

        params.kwargs_c_agb = dict(t_D=t_D, tau_agb=tau_agb, y0=y0, zeta=zeta_agb)
        params.alpha_c_agb = 1
    else:
        params.kwargs_c_agb = dict(mass_factor=mass_factor, no_negative=no_negative, interp_kind=interp_kind, low_z_flat=low_z_flat)

        y_c_agb = Y_C_AGB[agb_model]
        if alpha_agb is None:
            alpha_agb = y0 / y_c_agb

        y0 = alpha_agb * y_c_agb

        zeta_agb = ZETA_C_AGB[agb_model] * alpha_agb
        params.alpha_c_agb = alpha_agb


    return y0, zeta_agb


In [9]:
def enhance_fe_ia(params, factor):
    """if factor != 1, then enhances sne ia fe yield by factor but maintains same total fe yield, applied to params"""
    if factor == 1:
        return
    y_fe = params.y_fe_cc + params.y_fe_ia
    params.y_fe_ia *= factor
    params.y_fe_cc = y_fe - params.y_fe_ia
    return params


In [10]:
print_yc_tot()

total c yield


NameError: name 'C_AGB_Model' is not defined